# Phase de Nettoyage des données

Les différentes données récupérées ont tout d'abord besoin d'être nettoyées pour les besoins du projet. Si les données fournies par l'EUDA sont déjà très propres et ne nécéssiteront qu'un léger travail de tri, voire éventuellement quelques merge utiles sur certaines tables, ce n'est pas le cas des données scrapées à partir de pages Wikipedia qui vont nous demander un travail de nettoyage plus approfondi.

## Nettoyage des données scrapées depuis Wikipedia

On commence par générer notre dataframe à partir du scraping des données de [l'article Wikipedia sur la légalisation du Cannabis par Pays](https://en.wikipedia.org/wiki/Legality_of_cannabis). Le code détaillé et commenté se trouve dans le fichier `scraping.py`, on se contente ici d'appeler nos fonctions.


In [1]:
from Data.scraping import scraping as s

url = "https://en.wikipedia.org/wiki/Legality_of_cannabis"

table = s.scrape_second_wikipedia_table(url)
data, header = s.table_to_list(table)
legal_pays = s.make_pandas(data, header)

legal_pays.head()

,Recreational,Medical,Notes
Afghanistan,Illegal,Illegal,Main article: Cannabis in Afghanistan Producti...
Albania,Illegal,Legal,Main article: Cannabis in Albania Prohibited b...
Algeria,Illegal,Illegal,Main article: Cannabis in Algeria
Andorra,Illegal,Illegal,Main article: Cannabis in Andorra
Angola,Illegal,Illegal,Main article: Cannabis in Angola


On remarque alors que notre Dataframe contient plusieurs éléments qui nécéssitent d'être nettoyés, modifiés, voire supprimés. Dans l'ordre on peut citer:

1. **Supprimer la dernier colonne concernant les "Notes"**, qui peuvent nous être utile à des fins d'interprétation, ou encore pour aider à la classification ou à l'interprétation plus tard, mais qui en l'état seront inutiles à la statistique descriptive ou à la construction de modèles plus tard.

2. **Eliminer de notre dataframe les pays non-européens**, puisque les données de l'EUDA ne s'intéresse qu'au cas européeen, et que c'est là le coeur de notre sujet.

3. Enfin, il nous faudra **regrouper les différentes états des variables sous un format clair** En l'état, on dispose pour chaque pays d'une description parfois unique et idividuelle de la législation dans le pays donné. L'idéal serait de ne garder qu'une variable binaire: Légal/Illégal. Cependant la réalité est plus complexe dans certains pays, et il nous faudra alors faire des choix, et les justifier, pour essayer de ne garder qu'un nombre limité d'état pour chaque variable.

### Suppression de la colonne des Notes

Cela n'est pas bien compliqué, on utilise simplement les outils de la librairie Pandas:

In [2]:
import pandas as pd

legal_pays_clean = legal_pays.drop(columns = "Notes")

legal_pays_clean.head()

,Recreational,Medical
Afghanistan,Illegal,Illegal
Albania,Illegal,Legal
Algeria,Illegal,Illegal
Andorra,Illegal,Illegal
Angola,Illegal,Illegal


### Tri pour ne garder que les pays Européens

Cette opération n'est elle aussi pas très compliquée, on va se servir d'une table de correspondance scrapée sur le [site national de Statistiques Candiennes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110) (ce afin d'avoir le nom des pays en anglais), qui nous sera d'ailleurs aussi utile plus tard afin de faire correspondre chaque pays de l'Union européenne avec son code ISO 3316.

In [3]:
from Data.scraping import scraping as s

url = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
table_corres = s.make_pandas(data)

table_corres.head()

,0,1
Belgium,BE,BEL
Bulgaria,BG,BGR
Croatia,HR,HRV
Cyprus,CY,CYP
Czechia,CZ,CZE


On doit néanmoins ajouter à la main certains pays non-membres de l'Union Européenne, mais présent dans les données que nous avons à disposition (souvent des pays candidats comme la Turquie):

In [4]:
import pandas as pd

table_corres.loc["Austria"] = ["AT","AUT"]
table_corres.loc["Türkiye"] = ["TR","TUR"]
table_corres.loc["Norway"] = ["NO","NOR"]

table_corres.tail()

,0,1
Spain,ES,ESP
Sweden,SE,SWE
Austria,AT,AUT
Türkiye,TR,TUR
Norway,NO,NOR


On effectue ensuite le tri des pays européens selon la manière suivante, en supprimant de la table principale toutes les lignes qui n'ont pas d'équivalent dans la table de correspondance.

In [5]:
# On renomme certains pays pour correspondre aux données de l'EUDA 
legal_pays_clean.rename(index={"Turkey":"Türkiye"}, inplace=True)
legal_pays_clean.rename(index={"Czech Republic":"Czechia"}, inplace=True)

for index in legal_pays_clean.index:
    if index in table_corres.index:
        pass
    else:
        legal_pays_clean = legal_pays_clean.drop(index = index)

legal_pays_clean.head()


,Recreational,Medical
Austria,Possession for personal use decriminalized as ...,"Dronabinol, Sativex, Nabilone[25]"
Belgium,Decriminalized up to 3 g (1⁄10 oz) or cultivat...,Sativex[31]
Bulgaria,Illegal,Illegal
Croatia,Decriminalized[71],Legal[72]
Cyprus,Illegal,Legal[73]


### Recodage de chaque élément dans une variable ternaire

Il nous faut désormais faire un travail de recodage des informations contenues dans le tableau, afin de ne garder au final qu'une variable avec trois états possibles:

- `Legal` si la consommation, la possession et la vente sont légales et encadrées dans le pays
- `Illegal` si la consommation, la possession et la vente sont illégale et passibles de poursuites judiciaires
- `Decriminalized` si la consommation et la possession sont décriminalisées, mais la vente est illégale. On ne fait pas ici de différence entre la pénalisation ou non du cannabis (la seule différence étant que dans le cas ou ce dernier est pénalisé, la consommation et la possession sont toujours passibles d'une amende, mais pas de poursuites légales)

L'information est souvent contenue en clair dans notre tableau, mais perdue au milieu de précisions et de nuances propres à chaque pays. Par souci de simplicité, on appliquera les règles suivantes afin de recoder efficacement le tableau:

- **On ignorera toute limite de poids ou de nombres de plants**: Par exemple dans le cas de la République Tchèque, la possession de cannabis est décriminalisée jusqu'à 10g, et illégale au delà. On la considère donc simplement comme décriminalisée.

- **Dans le domaine médical, l'autorisation du Sativex vaut pour légalisation, celle du CBD non**: En effet, le Sativex contient du THC tout comme certains autres médicaments dérivés de cannabis, ce qui n'est pas le cas pour le CBD (ou alors en quantité très faible). En outre, au vu du fait que le CBD soit légal dans la plupart des pays, considérer son autorisation comme valant légalisation aurait pour conséquence d'appauvrir largement nos données.

- **On se tiendra à la stricte législation théorique, sans prendre en compte pour l'instant son application pratique**, cela aussi par souci de simplicité car la situation de chaque pays est unique et complexe. Par exemple, en Finlande, la possession et la consommation de cannabis est illégale *stricto sensu* et peut valoir une peine de prison. En réalité, les consommateurs s'en tirent souvent avec une simple amende; ce qui serait presque équivalent *en pratique* à une décriminalisation. Le fait est qu'il nous est impossible de trancher pour chaque pays à quel moment s'arrête la théorie et commence la pratique. Nous n'avons en effet ni la compétence sur le sujet, ni l'autorité nécessaire pour effectuer ce genre d'arbitrage, ce qui nous amène à ne prendre en compte que la législation en vigueur, et non son application réelle.
En revanche, il pourra nous être utile de tenir compte de ces différences d'application de la loi lors de l'analyse que nous ferons de nos données, mais pas dans la construction de celles-ci.

En prenant cela en compte, nous pouvons recoder le tableau de la manière suivante:

In [6]:
# Recodage des information selon les règles fixées plus haut

for index in legal_pays_clean.index:

    content = legal_pays_clean.loc[index].values.flatten().tolist()
    
    for i in range(len(content)):
        if "Legal" in content[i] or "Sativex" in content[i]:
            content[i] = "Legal"
        elif "Illegal" in content[i] or "CBD" in content[i]:
            content[i] = "Illegal"
        elif "Decriminalized" in content[i] or "decriminalized" in content[i]:
            content[i] = "Decriminalized"

    legal_pays_clean.loc[index] = content

print(legal_pays_clean)

               Recreational                                            Medical
Austria      Decriminalized                                              Legal
Belgium      Decriminalized                                              Legal
Bulgaria            Illegal                                            Illegal
Croatia      Decriminalized                                              Legal
Cyprus              Illegal                                              Legal
Czechia      Decriminalized                                              Legal
Denmark             Illegal                                              Legal
Estonia      Decriminalized                          With a special permit[88]
Finland             Illegal                                              Legal
France              Illegal  A two-year trial program involving 3,000 patie...
Germany               Legal                                              Legal
Greece              Illegal                         

Puisqu'il reste après filtrage encore quelques valeurs qui n'ont pas de définition claire, nous n'avons d'autre choix que de les recoder à la main, en justifiant bien évidemment nos choix à chaque étape:

- En Estonie, l'usage du cannabis pour la médecine est autorisé avec "un permis spécial". Dans les faits, une recherche plus détaillée sur ce permis nous apprend qu'il n'est en réalité jamais délivré et qu'un seul patient a ce jour a été autorisé à recevoir du Sativex. Cependant, pour être cohérent avec notre règle fixée plus haut de ne prendre en compte que la législation théorique en vigueur, on choisit de considérer l'usage comme Légal, puisqu'il est théoriquement possible.

- En France, l'usage cannabis thérapeutique est aujourd'hui sujet d'expérimentation dans certains cas précis et très encadrés, mais reste en théorie illégal. On choisit donc de le coder comme étant illégal puisque les expérimentations encadrées n'ouvrent pas à une possibilité d'utilisation pour la population générale.

- En Roumanie, l'usage thérapeutique de dérivés de cannabis avec une teneur en THC inférieure à 0,2% est autorisée. Etant donnée que la plupart des pays (dont la France) autorisent le CBD à condition qu'il ne contienne pas plus de 0,3% de THC, on choisit de considérer l'usage thérapeutique en Roumanie comme illégal, puisqu'on s'est fixé comme règle plus haut que la légalisation du CBD ne valait pas celle du cannabis.

In [7]:
# Recodage à la main des informations plus ambigues

legal_pays_clean.loc['France','Medical'] = 'Illegal'
legal_pays_clean.loc['Estonia','Medical'] = 'Legal'
legal_pays_clean.loc['Romania','Medical'] = 'Illegal'

legal_pays_clean.head()

,Recreational,Medical
Austria,Decriminalized,Legal
Belgium,Decriminalized,Legal
Bulgaria,Illegal,Illegal
Croatia,Decriminalized,Legal
Cyprus,Illegal,Legal


On obtient donc finalement un tableau complètement nettoyé et exploitable pour nos analyses futures !

# Ajout de nouvelles tables
## Importation des tables
On ajoute ici de nouvelles tables provenant elles aussi de www.euda.europa.eu.
Celles-ci serviront à la fois à produire des **statistiques descriptives** mais également à nourrir notre modèle de **machine learning**.

On commence par l'importation des données **générales** sur les **soins et traitements** liés à la consommation de drogue.
On utilise le module **Pandas** pour passer de fichier .xslx (Excel) à des tables Pandas.
Par ailleurs, on ne garde à chaque fois que les colonnes qui nous intéressent en vue d'un **Merge**.

In [8]:
import pandas as pd
import openpyxl


### On commence par les donnees generales
total = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/drogues_entrants.xlsx"
df1 = pd.read_excel(total, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df1 = df1[colonnes]


### On procede de la meme maniere pour les donnees concernant les personnes deja traitees auparavant
deja = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/deja_traite_entrants.xlsx"
df2 = pd.read_excel(deja, header=3)


# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df2 = df2[colonnes]




### On procede de la meme maniere pour les donnees concernant les personnes jamais traitees auparavant
jamais = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/jamais_traite_entrants.xlsx"
df3 = pd.read_excel(jamais, header=3)


# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df3 = df3[colonnes]


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Premier Merge : table_base
Une fois les tables préparées, on les **Merge** pour créer une unique table : *table_base*.
Celle-ci nous servira de base pour effectuer un **Merge** de plus grandes ampleurs afin d'obtenir une grande table compilant toutes les données disponibles ici sur le sujet des **soins et traiements** liés à la consommation de drogues.

In [9]:
### On effectue le merge (left join) afin de grouper les données par pays


## On commence par un merge "intermédiaire"
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df2 = df2.drop(columns=["Year of Treatment"])
intermediaire = pd.merge(df1, df2, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
intermediaire = intermediaire.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances_x": "All substances (global)",
    "Cannabis_x": "Cannabis (global)",
    "All substances_y": "All substances (deja traite.es)",
    "Cannabis_y": "Cannabis (deja traite.es)"
})


## On reproduit les mêmes étapes mais en "mergeant" intermediaire et df3 pour finaliser cette table de BaseException
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df3 = df3.drop(columns=["Year of Treatment"])
table_base = pd.merge(intermediaire, df3, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
table_base = table_base.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances (global)": "All substances (global)",
    "Cannabis (global)": "Cannabis (global)",
    "All substances (deja traite.es)": "All substances (deja traite.es)",
    "Cannabis (deja traite.es)": "Cannabis (deja traite.es)",
    "All substances": "All substances (jamais traite.es)",
    "Cannabis": "Cannabis (jamais traite.es)",
})

# On enlève les lignes en trop à la fin (vides ou inutiles)
table_base = table_base.drop(table_base.index[29:32:])


## Vérifications préalables au Merge final
Avant de **Merge** toutes les tables disponibles sur le sujet, on vérifie que la colonne **"Country"** sur la base de laquelle se fera le Merge existe bien dans tous les *Dataframes*.

In [10]:
# Etape préalable : avant d'effectuer le merge, on vérifie que la colonne 'Country' existe bien pour toutes les tables


### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os


### On commence par supprimer des tables potentiellement en double
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants"


# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]


# On sépare en deux listes les fichier ayant et n'ayant pas de colonne nommée "Country"
Liste_1 = []
Liste_2 = []
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier, header=3)
    if "Country" in df.columns:  
        Liste_1.append(chemin_fichier)
    else :
        Liste_2.append(chemin_fichier)
print(len(Liste_1), len(Liste_2))
# On a bien 39 tables dans la liste_1 et 0 dans Liste_2 : On est prêt.es pour le Merge !

/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

36 0


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Etape préalable n°2 : supprimer les doublons (en matière de *Dataframe*)
On utilise le module **os** pour vérifier dans le titre des fichiers si apparaît le terme **"Copie"** qui permet donc, le cas échéant, de supprimer des fichiers en doublon.
L'existence de tels fichiers provient d'erreurs humaines ayant eu lieu lors du téléchargement des bases de données sur le site www.euda.europa.eu.

In [11]:
# Etape préalable : avant d'effectuer le merge, on supprime les tables en double (triple, etc)


### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os


### On commence par supprimer des tables potentiellement en double
# On crée un dossier tampon qui va conserver nos tables modifiées
# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if "Copie" in f and f.endswith('.xlsx')]
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    if os.path.isfile(chemin_fichier):  # Vérifie si c'est bien un fichier
        os.remove(chemin_fichier)  # Supprime le fichier

## Etape préalable n°3 : supprimer la colonne Year of Treatment
Pourquoi un tel choix ? Cette colonne apparaît dans chaque table et donne des informations redondantes. Elle est, sauf très rare exception, identique dans chacune des *Dataframes*. Ainsi, afin d'éviter d'alourdir la table finale et de faciliter le Merge (**problème avec les noms de colonnes redondants**), on fait le choix de supprimer de toutes les tables (**sauf pour table_base**) la colonne **"Year of Treatment"**.

In [12]:
### On va supprimer les colonnes "Year of Treatment" présentent dans toutes les tables pour éviter les doublons dans notre table finale
# A quelques exceptions prêt négligeables, ka date est toujours 2022, l'information est donc redondante
fichiers2 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]
# Chemin du nouveau dossier
chemin_dossier_nouveau = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"

# Créer le dossier s'il n'existe pas
os.makedirs(chemin_dossier_nouveau, exist_ok=True)

for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier1 = os.path.join(chemin_dossier_nouveau, fichier)
    df = pd.read_excel(chemin_fichier, header=3)
    if "Year of Treatment" in df.columns :
        df = df.drop(columns=["Year of Treatment"])
    df.to_excel(chemin_fichier1, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

Etape préalable n°4 : 

In [13]:
### On commence par ajouter le suffixe ".deja" aux colonnes des tables tables concernées
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"


# On sélectionne les fichiers excel qui ont "deja" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "deja" in f and f.endswith('.xlsx')]
fichiers = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".deja" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien




### "jamais"
# On sélectionne les fichiers excel qui ont "jamais" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "jamais" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".jamais" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien


### "global"
# On sélectionne les fichiers excel qui ont "global" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "global" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".global" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien


test = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon/age_traitement_global.xlsx"
test = pd.read_excel(test)




## Etape préalable 4bis :


**Statut pro**

*.pro*

In [14]:
import pandas as pd
import os
dossier_2 = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
# On sélectionne les fichiers excel qui ont "pro" dans leur nom
fichiers1 = [f for f in os.listdir(dossier_2) if "pro" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".pro" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien



**sexe**

*.sex*

In [15]:
# On sélectionne les fichiers excel qui ont "sex" dans leur nom
fichiers2 = [f for f in os.listdir(dossier_2) if "sex" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".sex" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


**sdf**

*.sdf*

In [16]:
# On sélectionne les fichiers excel qui ont "sex" dans leur nom
fichiers3 = [f for f in os.listdir(dossier_2) if "sdf" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".sdf" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Niveau d'éducation**

*.lvl_educ*

In [17]:
# On sélectionne les fichiers excel qui ont "niveau" dans leur nom
fichiers4 = [f for f in os.listdir(dossier_2) if "niveau" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers4:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".lvl_educ" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Fréquence de consommation**

*.frqce_conso*

In [18]:
# On sélectionne les fichiers excel qui ont "frequence" dans leur nom
fichiers5 = [f for f in os.listdir(dossier_2) if "frequence" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".frqce_conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Nombre d'enfants**

*.enfants*

In [19]:
# On sélectionne les fichiers excel qui ont "enfants" dans leur nom
fichiers6 = [f for f in os.listdir(dossier_2) if "enfants" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers6:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".enfants" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Composition du ménage**

*.menage*

In [20]:
# On sélectionne les fichiers excel qui ont "menage" dans leur nom
fichiers7 = [f for f in os.listdir(dossier_2) if "menage" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".menage" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Age de l'entrant**

*.age_trait*

In [21]:
# On sélectionne les fichiers excel qui ont "traitement" dans leur nom
fichiers8 = [f for f in os.listdir(dossier_2) if "traitement" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers8:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".age_trait" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Age lors de la première consommation**

*.age_premiere_conso*

In [22]:
# On sélectionne les fichiers excel qui ont "premier" dans leur nom
fichiers9 = [f for f in os.listdir(dossier_2) if "premier" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers9:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".age_premiere_conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**De qui a émané la demande de soins**
(institution judiciaire, proches, personne concernée, *etc*)

*.demande*

In [23]:
# On sélectionne les fichiers excel qui ont "demande" dans leur nom
fichiers10 = [f for f in os.listdir(dossier_2) if "demande" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers10:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".demande" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

In [24]:
### Maintenant que ces modifications sont faites, on effectue le merge
# On effectue autant de merge que nécessaire en créant de nouveaux suffixes à chaque fois
test=table_base
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
for fichier in fichiers1 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers2 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers3 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers4 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers5 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers6 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers7 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers8 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers9 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers10 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
soins_cannabis = test
soins_cannabis

,Country,Year of Treatment,All substances (global),Cannabis (global),All substances (deja traite.es),Cannabis (deja traite.es),All substances (jamais traite.es),Cannabis (jamais traite.es),Occasionally employed.jamais.pro,Regularly employed.jamais.pro,...,Total.deja.demande,Court / probation / police.jamais.demande,General practitioner.jamais.demande,Other drug treatment centre.jamais.demande,"Other health, medical or social service.jamais.demande",Educational services.jamais.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.jamais.demande",Other.jamais.demande,Not known / missing.jamais.demande,Total.jamais.demande
0,Austria,2022.0,4709.0,1124.0,2756.0,458.0,1953.0,666.0,8.0,275.0,...,458.0,179.0,15.0,11.0,190.0,16.0,209.0,39.0,7.0,666.0
1,Belgium,2022.0,12144.0,3501.0,7376.0,1524.0,4151.0,1850.0,90.0,521.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,2022.0,1879.0,109.0,1164.0,42.0,605.0,66.0,2.0,9.0,...,42.0,12.0,0.0,2.0,2.0,0.0,41.0,9.0,0.0,66.0
3,Croatia *,2022.0,2315.0,478.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cyprus,2022.0,884.0,340.0,308.0,66.0,535.0,251.0,11.0,85.0,...,66.0,124.0,0.0,6.0,7.0,1.0,62.0,48.0,3.0,251.0
5,Czechia,2020.0,7283.0,1072.0,3336.0,469.0,2519.0,539.0,24.0,71.0,...,469.0,33.0,38.0,12.0,91.0,9.0,319.0,2.0,35.0,539.0
6,Denmark,2022.0,8241.0,3085.0,4408.0,1473.0,3368.0,1493.0,167.0,319.0,...,1473.0,56.0,18.0,4.0,82.0,50.0,1123.0,146.0,14.0,1493.0
7,Estonia,2022.0,417.0,26.0,290.0,15.0,124.0,11.0,2.0,2.0,...,15.0,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,11.0
8,Finland,2021.0,427.0,62.0,263.0,26.0,164.0,36.0,2.0,4.0,...,26.0,2.0,0.0,5.0,9.0,5.0,13.0,0.0,2.0,36.0
9,France,2021.0,43078.0,19758.0,10628.0,3647.0,18195.0,10680.0,1078.0,2732.0,...,6851.0,2587.0,232.0,90.0,803.0,150.0,2066.0,137.0,439.0,6504.0


# Ajout de nouvelles tables : infractions à la loi

Ici, on va procéder de manière analogue à la partie précédente sur l'ajout des tables en matière de soins et de traitements liés à la consommation de drogues en Europe.

## Importation des tables
On utilise les modules **Pandas** et **OS** afin de passer les fichiers du format .xslx (Excel) au format Pandas.

## Vérification du type de fichier
On a rencontré au cours de notre travail des problèmes à causes de tables téléchargées au mauvais format, ce qui empêchait le code de tourner correctement.
Pour éviter de rencontre à nouveau ce type de souci, on vérifie que tous les fichiers sont bien des *.xlsx*.

In [25]:
import os
liste = []
### On crée une fonction pour récupérer le type d'un fichier basé sur son extension grâce au module os
def get_file_type(file_path):
    _, file_extension = os.path.splitext(file_path)
    return file_extension.lower()  # On met en minuscule pour éviter les problèmes



### On ajoute le type de chaque fichier à une liste
fichiers7 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    liste.append(get_file_type(fichier))

### On fait un test pour vérifier que tous les fichiers sont bien de .xlsx
all_equal = all(x == liste[0] for x in liste)
print(all_equal)

True


## On spécifie le type de drogue concernée

L'idée est ici de faire en sorte de rendre les tables plus **lisibles**, en particulier en vue d'un **merge** de celles-ci.
Le nom des colonnes étant redondant et peu lisible sans le nom de la table (2010, 2011, *etc*), l'idée est de donner via les **noms  de colonnes** des **clés de lecture**.

Pour cela, nous allons ajouter des informations au nom de colonne *via* le format suivant :
**année.détail1.détail2**



Voici la classification choisie :

**Détail1** :
Regroupe les différents types de drogues étudiées (glbal (*i.e.* toutes les drogues), cannabis (*i.e.* sous toutes ses formes), herbe, raisine, huile et plante).

**Détail2**
Regroupe les différents types d'infractions (vente, consommation, production, infractions/délinquants (total des différents types d'infractions)).

**Cas global**

In [26]:
import os
import pandas as pd

### On commence par ajouter le suffixe ".cannabis" aux colonnes des tables concernées
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Delits_Crimes"
dossier_2 = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
fichier = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]

# On sélectionne les fichiers excel qui ont "Cannabis" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "Cannabis" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".cannabis" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Herbe**

In [27]:
### "Herbe"
# On sélectionne les fichiers excel qui ont "Herbe" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "Herbe" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".herbe" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien



/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Huile**

In [28]:
### "Huile"
# On sélectionne les fichiers excel qui ont "Huile" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "Huile" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".huile" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Plante**

In [29]:
### "Plante"
# On sélectionne les fichiers excel qui ont "Plante" dans leur nom
fichiers5 = [f for f in os.listdir(dossier) if "Plante" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".plante" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Raisine**

In [30]:
### "Raisine"
# On sélectionne les fichiers excel qui ont "Raisine" dans leur nom
fichiers6 = [f for f in os.listdir(dossier) if "Raisine" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers6:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".raisine" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Global**

In [31]:
### "Global"
# On sélectionne les fichiers excel qui ont "global" dans leur nom
fichiers7 = [f for f in os.listdir(dossier) if "Global" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".global" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## On spécifie le type d'infractions observées

**Consommation**

In [32]:
# On sélectionne les fichiers excel qui ont "Consommation" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "Consommation" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Délinquants**

In [33]:
### "Delinquants"
# On sélectionne les fichiers excel qui ont "Delinquants" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "Deliquants" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".delinquants" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Infractions**

In [34]:
### "Infractions"
# On sélectionne les fichiers excel qui ont "Infractions" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "Infractions" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".infractions" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Production**

In [35]:
### "Production"
# On sélectionne les fichiers excel qui ont "Production" dans leur nom
fichiers4 = [f for f in os.listdir(dossier) if "Production" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers4:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".prod" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Ventes**

In [36]:
### "Ventes"
# On sélectionne les fichiers excel qui ont "Ventes" dans leur nom
fichiers5 = [f for f in os.listdir(dossier) if "Ventes" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".ventes" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

## On passe au Merge final

On effectue un merge sur toutes les tables concernant les infractions à la loi ayant trait à la drogue (consommation, production, etc).
Le fait d'avoir modifier le nom des colonnes permet de donner un résultat plus lisible.

In [37]:
### Maintenant que ces modifications sont faites, on effectue le merge
# On effectue autant de merge que nécessaire en créant de nouveaux suffixes à chaque fois
test= pd.read_excel("/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon/Nombre_Consommation_Cannabis.xlsx")
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
for fichier in fichiers1 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers2 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers3 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers4 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers5 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
table_infractions_legales = test
table_infractions_legales

,Country,2022.0.cannabis.conso_x,2021.0.cannabis.conso_x,2020.0.cannabis.conso_x,2019.0.cannabis.conso_x,2018.0.cannabis.conso_x,2017.0.cannabis.conso_x,2016.0.cannabis.conso_x,2015.0.cannabis.conso_x,2014.0.cannabis.conso_x,...,2016.0.herbe.ventes,2015.0.herbe.ventes,2014.0.herbe.ventes,2013.0.herbe.ventes,2012.0.herbe.ventes,2011.0.herbe.ventes,2010.0.herbe.ventes,2009.0.herbe.ventes,2008.0.herbe.ventes,2007.0.herbe.ventes
0,Austria,19909.0,19919.0,22778.0,24618.0,24680.0,28890.0,23977.0,20456.0,21272.0,...,6295.0,6100.0,5832.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Belgium,29572.0,28857.0,31540.0,31688.0,30148.0,29916.0,27080.0,28237.0,29825.0,...,4581.0,4591.0,4689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3465.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Croatia,5159.0,5080.0,4810.0,5689.0,6788.0,6253.0,5493.0,4268.0,4866.0,...,1483.0,1260.0,1128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cyprus,563.0,512.0,606.0,692.0,862.0,703.0,643.0,690.0,799.0,...,65.0,46.0,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Czechia,6443.0,5253.0,4195.0,5293.0,6176.0,5019.0,4744.0,4224.0,500.0,...,899.0,778.0,734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Estonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Finland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Création de tables simplifiées
On crée deux tables simplifiées ne gardant que les informations sur le **cannabis** et les **drogues en général** et plus sur les **différents types de cannabis** (raisine, herbe, *etc*).

On fait ça pour les deux tables "finales" **table_infractions_legales** et pour **soins_cannabis**.

# Pour la table concernant les infractions légales
On se sert des spécifications dans le **nom des colonnes *.detail1***.
Ainsi, on va donc uniquement sélectionner les colonnes contenant "**.cannabis**" et "**.global**".

In [38]:
# Sélection des colonnes contenant ".cannabis" et ".global"
table_infractions_simple = table_infractions_legales[[col for col in table_infractions_legales.columns if ".cannabis" in col or ".global" in col or col == "Country"]]
table_infractions_simple

,Country,2022.0.cannabis.conso_x,2021.0.cannabis.conso_x,2020.0.cannabis.conso_x,2019.0.cannabis.conso_x,2018.0.cannabis.conso_x,2017.0.cannabis.conso_x,2016.0.cannabis.conso_x,2015.0.cannabis.conso_x,2014.0.cannabis.conso_x,...,2016.0.cannabis.ventes,2015.0.cannabis.ventes,2014.0.cannabis.ventes,2013.0.cannabis.ventes,2012.0.cannabis.ventes,2011.0.cannabis.ventes,2010.0.cannabis.ventes,2009.0.cannabis.ventes,2008.0.cannabis.ventes,2007.0.cannabis.ventes
0,Austria,19909.0,19919.0,22778.0,24618.0,24680.0,28890.0,23977.0,20456.0,21272.0,...,7372.0,7086.0,6705.0,1595.0,1409.0,1428.0,1454.0,1460.0,1261.0,1833.0
1,Belgium,29572.0,28857.0,31540.0,31688.0,30148.0,29916.0,27080.0,28237.0,29825.0,...,6316.0,6460.0,6637.0,6074.0,5942.0,8341.0,8574.0,8892.0,9047.0,8200.0
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3465.0,...,NaN,NaN,790.0,1424.0,800.0,625.0,627.0,641.0,438.0,314.0
3,Croatia,5159.0,5080.0,4810.0,5689.0,6788.0,6253.0,5493.0,4268.0,4866.0,...,1719.0,1620.0,1399.0,1799.0,1442.0,1790.0,1746.0,1519.0,931.0,1199.0
4,Cyprus,563.0,512.0,606.0,692.0,862.0,703.0,643.0,690.0,799.0,...,106.0,142.0,88.0,127.0,124.0,88.0,117.0,129.0,NaN,128.0
5,Czechia,6443.0,5253.0,4195.0,5293.0,6176.0,5019.0,4744.0,4224.0,500.0,...,1249.0,1029.0,1052.0,1073.0,870.0,733.0,632.0,661.0,608.0,463.0
6,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Estonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Finland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5967.0,NaN,NaN


On est passé de **417** colonnes à **160 colonnes**.

# Pour la table concernant les soins

On se sert des spécifications dans le **nom des colonnes *.detail***.
Ainsi, on va donc uniquement sélectionner les colonnes contenant "**global**. Pas juste **".global**" pour ne pas exclure les colonnes "All Substances (global)" et "Cannabis (global)".

In [39]:
# Sélection des colonnes contenant ".global"
table_soins_simple = soins_cannabis[[col for col in soins_cannabis.columns if "global" in col or col == "Country" or col == "Year of Treatment"]]

# On affiche le résultat pour vérifier que cela a correctment fonctionné
table_soins_simple

,Country,Year of Treatment,All substances (global),Cannabis (global),Occasionally employed.global.pro,Regularly employed.global.pro,Student.global.pro,Unemployed / discouraged.global.pro,Receiving social benefits / pensioners / house-makers / disable.global.pro,Other.global.pro,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022.0,4709.0,1124.0,13.0,440.0,75.0,287.0,40.0,17.0,...,1124.0,337.0,23.0,30.0,275.0,19.0,353.0,54.0,33.0,1124.0
1,Belgium,2022.0,12144.0,3501.0,142.0,949.0,776.0,652.0,723.0,50.0,...,3501.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,2022.0,1879.0,109.0,4.0,19.0,21.0,53.0,11.0,1.0,...,109.0,28.0,1.0,4.0,3.0,0.0,63.0,9.0,1.0,109.0
3,Croatia *,2022.0,2315.0,478.0,53.0,102.0,145.0,135.0,7.0,8.0,...,478.0,119.0,20.0,0.0,95.0,NaN,197.0,32.0,15.0,478.0
4,Cyprus,2022.0,884.0,340.0,17.0,116.0,77.0,98.0,5.0,16.0,...,340.0,139.0,0.0,12.0,10.0,1.0,104.0,68.0,6.0,340.0
5,Czechia,2020.0,7283.0,1072.0,50.0,138.0,82.0,215.0,32.0,6.0,...,1072.0,77.0,66.0,44.0,206.0,11.0,600.0,7.0,61.0,1072.0
6,Denmark,2022.0,8241.0,3085.0,293.0,571.0,307.0,86.0,1256.0,398.0,...,3085.0,163.0,32.0,24.0,138.0,64.0,2344.0,295.0,25.0,3085.0
7,Estonia,2022.0,417.0,26.0,2.0,7.0,1.0,14.0,2.0,NaN,...,26.0,NaN,NaN,1.0,NaN,NaN,25.0,NaN,NaN,26.0
8,Finland,2021.0,427.0,62.0,4.0,7.0,14.0,31.0,4.0,0.0,...,62.0,2.0,3.0,5.0,16.0,5.0,26.0,0.0,5.0,62.0
9,France,2021.0,43078.0,19758.0,2082.0,5041.0,3436.0,3110.0,3990.0,0.0,...,19759.0,7325.0,651.0,348.0,2073.0,390.0,7057.0,330.0,3012.0,21186.0


On est passé de **257** colonnes à **87 colonnes**.

# Valeurs manquantes : colonne par colonne

## Comptage des valeurs manquantes

On crée une **ligne** supplémentaire dans chaque *dataframe* qui affichera pour **chaque colonne** la valeur prise par un **fonction** qui compte les **valeurs manquantes**.

**Pourquoi procéder ainsi ?**
L'étape suivante consistera à choisir les variables (colonnes ou groupe de colonnes) pour lesquelles on a le moins de valeurs manquantes.

**NB : on en profite aussi pour convertir toutes les colonnes - sauf "Country" - en NOMBRE ENTIER afin de rendre la table plus lisible (en effet, les données sont soit des dates, soit des nombres de patients)**

## On commence par table_soins_simple

Tout d'abord, on s'occupe d'**uniformiser les valeurs manquantes et aberrantes** sous la forme "NaN".
Ainsi, on agit sur toutes les colonnes pour que "**inf**" et "**-inf**" s'affiche également comme NaN.

Ensuite, on fait en sorte que toutes les valeurs soient des **entiers (Int)** pour les raisons décrites précédemment.

In [40]:
for col in table_soins_simple.columns:
    if col != 'Country':  # On exclut la colonne 'Country'
        table_soins_simple[col] = pd.to_numeric(table_soins_simple[col], errors='coerce')  # Coerce transforme les valeurs non convertibles en NaN
        table_soins_simple[col] = table_soins_simple[col].astype('Int64')
        
# On affiche le dataframe pour vérifier que l'opération a fonctionné correctement
table_soins_simple

/tmp/ipykernel_58320/2913384960.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_soins_simple[col] = pd.to_numeric(table_soins_simple[col], errors='coerce')  # Coerce transforme les valeurs non convertibles en NaN
/tmp/ipykernel_58320/2913384960.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_soins_simple[col] = table_soins_simple[col].astype('Int64')


,Country,Year of Treatment,All substances (global),Cannabis (global),Occasionally employed.global.pro,Regularly employed.global.pro,Student.global.pro,Unemployed / discouraged.global.pro,Receiving social benefits / pensioners / house-makers / disable.global.pro,Other.global.pro,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022,4709,1124,13,440,75,287,40,17,...,1124,337,23,30,275,19,353,54,33,1124
1,Belgium,2022,12144,3501,142,949,776,652,723,50,...,3501,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Bulgaria,2022,1879,109,4,19,21,53,11,1,...,109,28,1,4,3,0,63,9,1,109
3,Croatia *,2022,2315,478,53,102,145,135,7,8,...,478,119,20,0,95,<NA>,197,32,15,478
4,Cyprus,2022,884,340,17,116,77,98,5,16,...,340,139,0,12,10,1,104,68,6,340
5,Czechia,2020,7283,1072,50,138,82,215,32,6,...,1072,77,66,44,206,11,600,7,61,1072
6,Denmark,2022,8241,3085,293,571,307,86,1256,398,...,3085,163,32,24,138,64,2344,295,25,3085
7,Estonia,2022,417,26,2,7,1,14,2,<NA>,...,26,<NA>,<NA>,1,<NA>,<NA>,25,<NA>,<NA>,26
8,Finland,2021,427,62,4,7,14,31,4,0,...,62,2,3,5,16,5,26,0,5,62
9,France,2021,43078,19758,2082,5041,3436,3110,3990,0,...,19759,7325,651,348,2073,390,7057,330,3012,21186


Ceci étant fait, on crée une **fonction de comptage des valeurs manquantes** qui s'affichera comme une nouvelle ligne pour chaque colonne (sauf "Country" et "Year of Treatment").

In [41]:
# On compte le nombre de NaN dans chaque colonne du dataframe "table_soins_simple"
Nb_valeurs_manquantes = table_soins_simple.isna().sum()

# On ajoute une ligne contenant cette variable au dataframe
table_soins_simple.loc['Nb_valeurs_manquantes'] = Nb_valeurs_manquantes

# On affiche le dataframe pour vérifier que l'opération a fonctionné correctement
table_soins_simple

/tmp/ipykernel_58320/4062901942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_soins_simple.loc['Nb_valeurs_manquantes'] = Nb_valeurs_manquantes


,Country,Year of Treatment,All substances (global),Cannabis (global),Occasionally employed.global.pro,Regularly employed.global.pro,Student.global.pro,Unemployed / discouraged.global.pro,Receiving social benefits / pensioners / house-makers / disable.global.pro,Other.global.pro,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022,4709,1124,13,440,75,287,40,17,...,1124,337,23,30,275,19,353,54,33,1124
1,Belgium,2022,12144,3501,142,949,776,652,723,50,...,3501,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Bulgaria,2022,1879,109,4,19,21,53,11,1,...,109,28,1,4,3,0,63,9,1,109
3,Croatia *,2022,2315,478,53,102,145,135,7,8,...,478,119,20,0,95,<NA>,197,32,15,478
4,Cyprus,2022,884,340,17,116,77,98,5,16,...,340,139,0,12,10,1,104,68,6,340
5,Czechia,2020,7283,1072,50,138,82,215,32,6,...,1072,77,66,44,206,11,600,7,61,1072
6,Denmark,2022,8241,3085,293,571,307,86,1256,398,...,3085,163,32,24,138,64,2344,295,25,3085
7,Estonia,2022,417,26,2,7,1,14,2,<NA>,...,26,<NA>,<NA>,1,<NA>,<NA>,25,<NA>,<NA>,26
8,Finland,2021,427,62,4,7,14,31,4,0,...,62,2,3,5,16,5,26,0,5,62
9,France,2021,43078,19758,2082,5041,3436,3110,3990,0,...,19759,7325,651,348,2073,390,7057,330,3012,21186


## Même procédé sur les valeurs manquantes pour table_infractions_simple :
1) On **uniformise** les valeurs manquantes/aberrantes
2) On crée une ligne qui affiche une **fonction de comptage des valeurs manquantes**

In [43]:
### 1) Uniformisation des valeurs manquantes/aberrantes
for col in table_infractions_simple.columns:
    if col != 'Country':  # On exclut la colonne 'Country'
        table_infractions_simple[col] = pd.to_numeric(table_infractions_simple[col], errors='coerce')  # Coerce transforme les valeurs non convertibles en NaN

### 2) On crée la nouvelle ligne
Nb_valeurs_manquantes = table_infractions_simple.isna().sum() # On compte le nombre de NaN dans chaque colonne du dataframe "table_infractions_simple"
table_infractions_simple.loc['Nb_valeurs_manquantes'] = Nb_valeurs_manquantes # On ajoute une ligne contenant cette variable au dataframe

# On affiche le dataframe pour vérifier que l'opération a fonctionné correctement
table_infractions_simple




/tmp/ipykernel_58320/2944171851.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_infractions_simple[col] = pd.to_numeric(table_infractions_simple[col], errors='coerce')  # Coerce transforme les valeurs non convertibles en NaN
/tmp/ipykernel_58320/2944171851.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_infractions_simple.loc['Nb_valeurs_manquantes'] = Nb_valeurs_manquantes # On ajoute une ligne contenant cette variable au dataframe


,Country,2022.0.cannabis.conso_x,2021.0.cannabis.conso_x,2020.0.cannabis.conso_x,2019.0.cannabis.conso_x,2018.0.cannabis.conso_x,2017.0.cannabis.conso_x,2016.0.cannabis.conso_x,2015.0.cannabis.conso_x,2014.0.cannabis.conso_x,...,2016.0.cannabis.ventes,2015.0.cannabis.ventes,2014.0.cannabis.ventes,2013.0.cannabis.ventes,2012.0.cannabis.ventes,2011.0.cannabis.ventes,2010.0.cannabis.ventes,2009.0.cannabis.ventes,2008.0.cannabis.ventes,2007.0.cannabis.ventes
0,Austria,19909.0,19919.0,22778.0,24618.0,24680.0,28890.0,23977.0,20456.0,21272.0,...,7372.0,7086.0,6705.0,1595.0,1409.0,1428.0,1454.0,1460.0,1261.0,1833.0
1,Belgium,29572.0,28857.0,31540.0,31688.0,30148.0,29916.0,27080.0,28237.0,29825.0,...,6316.0,6460.0,6637.0,6074.0,5942.0,8341.0,8574.0,8892.0,9047.0,8200.0
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3465.0,...,NaN,NaN,790.0,1424.0,800.0,625.0,627.0,641.0,438.0,314.0
3,Croatia,5159.0,5080.0,4810.0,5689.0,6788.0,6253.0,5493.0,4268.0,4866.0,...,1719.0,1620.0,1399.0,1799.0,1442.0,1790.0,1746.0,1519.0,931.0,1199.0
4,Cyprus,563.0,512.0,606.0,692.0,862.0,703.0,643.0,690.0,799.0,...,106.0,142.0,88.0,127.0,124.0,88.0,117.0,129.0,NaN,128.0
5,Czechia,6443.0,5253.0,4195.0,5293.0,6176.0,5019.0,4744.0,4224.0,500.0,...,1249.0,1029.0,1052.0,1073.0,870.0,733.0,632.0,661.0,608.0,463.0
6,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Estonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Finland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5967.0,NaN,NaN


# Lisibilité : on crée une table des différentes variables présentes dans les dataframes
*Les dataframes concernés sont table_infractions_simple et table_soins_simple*

**Comment va-t-on procéder ?**
On va créer une liste de chaque variable en subdivisant à l'aide de *.detail*

### Pour la table *table_infractions_simple*

In [ ]:
Var_infractions_1 = ["Consommation", "Infractions", "Production", "Ventes"]
Var_infractions_2 = ["Cannabis", "Global"]
# NB1 : il suffit de croiser les deux listes (ex : .cannabis.consommation)
# NB2 : pour avoir l'année (format : '20XX'), il faut rajouter en prefixe '20XX.0.'
### Exemple : 2012.0.Infractions.Cannabis
table_soins_simple

,Country,Year of Treatment,All substances (global),Cannabis (global),Occasionally employed.global.pro,Regularly employed.global.pro,Student.global.pro,Unemployed / discouraged.global.pro,Receiving social benefits / pensioners / house-makers / disable.global.pro,Other.global.pro,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022,4709,1124,13,440,75,287,40,17,...,1124,337,23,30,275,19,353,54,33,1124
1,Belgium,2022,12144,3501,142,949,776,652,723,50,...,3501,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Bulgaria,2022,1879,109,4,19,21,53,11,1,...,109,28,1,4,3,0,63,9,1,109
3,Croatia *,2022,2315,478,53,102,145,135,7,8,...,478,119,20,0,95,<NA>,197,32,15,478
4,Cyprus,2022,884,340,17,116,77,98,5,16,...,340,139,0,12,10,1,104,68,6,340
5,Czechia,2020,7283,1072,50,138,82,215,32,6,...,1072,77,66,44,206,11,600,7,61,1072
6,Denmark,2022,8241,3085,293,571,307,86,1256,398,...,3085,163,32,24,138,64,2344,295,25,3085
7,Estonia,2022,417,26,2,7,1,14,2,<NA>,...,26,<NA>,<NA>,1,<NA>,<NA>,25,<NA>,<NA>,26
8,Finland,2021,427,62,4,7,14,31,4,0,...,62,2,3,5,16,5,26,0,5,62
9,France,2021,43078,19758,2082,5041,3436,3110,3990,0,...,19759,7325,651,348,2073,390,7057,330,3012,21186


### Pour la table *table_soins_simple*

In [ ]:
Var_soins_1 = ['.pro','.sex', '.sdf','.lvl_educ','.frqce_conso','.enfants','.menage','.age_trait','.age_premiere_conso','.demande']
Var_soins_2 = ['cannabis', 'global']

### chaque élement de Var_soins_1 est une table de base qui contient elle même d'autres variables
# On va donc imprimer la liste de ces variables, il suffira donc de procéder par arborescence : on choisit le thème qu'on veut étudier dans la liste puis on regarde les variables disponibles à ce sujet
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé"
for j in Var_soins_1 : 
    fichiers = [f for f in os.listdir(dossier) if j in f and f.endswith('.xlsx')]
    # On ajoute le suffixe aux colonnes souhaitées
    for fichier in fichiers:
        df = pd.read_excel(chemin_fichier)

colonnes_par_soin = {}

for soin in Var_soins_1:
    colonnes = [col for col in table_soins_simple.columns if soin in col]
    colonnes_par_soin[soin] = colonnes

print (colonnes_par_soin)





{'.pro': ['Occasionally employed.global.pro', 'Regularly employed.global.pro', 'Student.global.pro', 'Unemployed / discouraged.global.pro', 'Receiving social benefits / pensioners / house-makers / disable.global.pro', 'Other.global.pro', 'Not known / missing.global.pro', 'Total.global.pro'], '.sex': ['All.global.sex', 'Males.global.sex', 'Females.global.sex', 'Not known / missing.global.sex'], '.sdf': ['Stable accommodation.global.sdf', 'Unstable accommodation and/or homeless.global.sdf', 'In detention.global.sdf', 'Other.global.sdf', 'Not known / missing.global.sdf', 'Total.global.sdf'], '.lvl_educ': ['Never went to school / never completed primary school (ISCED 0).global.lvl_educ', 'Primary level of education (ISCED 1).global.lvl_educ', 'Secondary level of education (ISCED 2 and ISCED 3).global.lvl_educ', 'Higher education (ISCED 4 to ISCED 6).global.lvl_educ', 'Not known / missing.global.lvl_educ', 'Total.global.lvl_educ'], '.frqce_conso': ['Daily.global.frqce_conso', '4 to 6 days p

# Dataframe sur les données macroéconomiques des pays de l'union européenne élargie
*On travaille sur des jeux de données issues de l'INSEE et d'EUROSTAT*

Ici, l'idée est de fournir un tableau général de la situation de chaque pays étudié. 
Les jeux de données que nous traitons concernent donc des variables à proprement **économiques** (PIB, Revenu médian par pays) mais également **socio-économiques** (IDH notamment).
Par ailleurs, on avons choisi de là encore prendre des données sur plusieurs années (*2011-2022*) afin de données du recul sur les observations, d'offrir plus de perspectives en matière de statistiques descriptives mais également d'avoir plus de données pour *nourrir* nos modèles de prédiction.

## Importation des données au format pandas

Les données sont initialement enregistrées au format **excel**, on utilise donc *pandas* pour les importer en **dataframes** et pouvoir travailler dessus.

#### ***Données sur le PIB***

In [15]:
import pandas as pd

# On importe au format pandas les données sur le chômage
pib = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/estat_tipsna40_filtered_en.csv')
pib.drop(columns=['DATAFLOW','LAST UPDATE','freq','na_item','unit','OBS_FLAG'],inplace=True)

pib = pib[pib['TIME_PERIOD'] == 2022]

pib.rename(columns={'geo':'Country','OBS_VALUE':'PIB_r_hab'}, inplace=True)

pib

,Country,TIME_PERIOD,PIB_r_hab
27,Austria,2022,42540
56,Belgium,2022,40180
85,Bulgaria,2022,8660
114,Cyprus,2022,28860
143,Czechia,2022,18450
172,Germany,2022,39820
201,Denmark,2022,54310
230,Euro area – 20 countries (from 2023),2022,34190
259,Estonia,2022,19240
288,Greece,2022,18310


#### ***Données sur le chômage***

In [48]:
import pandas as pd

# On importe au format pandas les données sur le chômage
chomage = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/estat_lfsa_ugadra_filtered_en.csv')
chomage.drop(columns=['DATAFLOW','LAST UPDATE','freq','age','sex','regis_es','duration','unit','OBS_FLAG'],inplace=True)

chomage.rename(columns={'geo':'Country','OBS_VALUE':'tx_chomage'}, inplace=True)

chomage_filtre = pd.DataFrame(columns=['Country','TIME_PERIOD','tx_chomage'])

# On fait en sorte de récupérer à chaque fois la donnée la plus récente disponible
for index in chomage.index:
    country = chomage.at[index, 'Country']
    last_buffer = buffer
    buffer = chomage.loc[index].tolist()
    chomage_filtre.loc[country] = buffer
    if pd.isna(chomage_filtre.at[country,'tx_chomage']) == True:
        chomage_filtre.loc[country] = last_buffer
        buffer = last_buffer

chomage_filtre.reset_index(inplace=True)
chomage_filtre.drop(columns='index', inplace=True)

chomage_filtre

,Country,TIME_PERIOD,tx_chomage
0,Austria,2023,42.7
1,Bosnia and Herzegovina,2023,24.7
2,Belgium,2023,16.1
3,Belgium,2023,16.1
4,Switzerland,2023,23.9
5,Cyprus,2021,22.0
6,Czechia,2023,31.9
7,Germany,2023,33.7
8,Denmark,2023,15.0
9,Euro area – 20 countries (from 2023),2023,28.4
